# Coursera Capstone Project 
##  FoursquareAPI  
### week3

Using BeautifulSoup to scrape data which is from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

First we need to install BeautifulSoup, lxml, html5lib and requests by using
$ pip install beautifulsoup4
$ pip install lxml
$ pip install htmls5lib
$ pip install requests

In [1]:
#scraping web by using BeautifulSoup
#also, save it to csv file
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd

with open('/home/hatai/Downloads/List of postal codes of Canada_ M - Wikipedia.html') as html_file:
    soup=BeautifulSoup(html_file,'lxml')

csv_file=open('toronto_postcode.csv','w')
csv_writer=csv.writer(csv_file)
csv_writer.writerow(['Postcode','Borough','Neighbourhood'])

table=soup.find('table')
table_rows=table.find_all('tr')

for tr in table_rows:
    td=tr.find_all('td')
    row=[i.text for i in td]
    csv_writer.writerow(row)

csv_file.close()

In [2]:
#Ignore cells with a borough that is Not assigned.
#More than one neighborhood can exist in one postal code area.
#These rows will be combined into one row with the neighborhoods separated with a comma.
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

import pandas as pd
import numpy as np

df=pd.read_csv(r"/home/hatai/toronto_postcode.csv")

df.drop(df[df['Borough']=='Not assigned'].index,axis=0,inplace=True)  


#new_df=df.reset_index(drop=True)

#group by 'Postcode' and 'Borough' then
#transform data from column 'Neighbourhood' and assign result to a new column 'D'
#column 'D' will contain a string of comma separated values from
#column 'Neighbourhood'
df['D']=df.groupby(['Postcode','Borough'])['Neighbourhood'].transform( lambda x: ','.join(x))

#remove column 'Neighbourhood'
df.drop("Neighbourhood",axis=1,inplace=True)

#rename new column 'D' to 'Neighbourhood'
df.rename(columns={"D":"Neighbourhood"},inplace=True)

#drop duplicate rows as the above transform put comma separated values
#in every row where 'Postcode' and 'Borough' are grouped.
df.drop_duplicates(subset ="Neighbourhood",keep = 'first', inplace = True)

#the index will point to the original data frame index values
#so redo the index for this new data frame.
df.reset_index(drop=True,inplace=True)
print(df.head())

#writing a pandas dataframe to CSV file
df.to_csv('Toronto_postcode.csv')


#checking 'Not assigned' in column 'Neighbourhood'
dfTolist=df['Neighbourhood'].tolist()
dfTolist.count('Not assigned\n')
#there is only one 'Not assigned' so I manually deleted it in csv file.


  Postcode           Borough                        Neighbourhood
0      M3A        North York                          Parkwoods\n
1      M4A        North York                   Victoria Village\n
2      M5A  Downtown Toronto         Harbourfront\n,Regent Park\n
3      M6A        North York  Lawrence Heights\n,Lawrence Manor\n
4      M7A      Queen's Park                       Not assigned\n


1

# Answer: First assighnment, Check the size of dataframe by using .shape

In [3]:
df.shape

(103, 3)

# Prepare and clean data

In [4]:
#remove \n in dataframe
df=pd.read_csv('Toronto_postcode.csv')
keep_col=['Postcode','Borough','Neighbourhood']
df = df[keep_col].replace('\n',' ', regex=True)
print(df.head())

  Postcode           Borough                      Neighbourhood
0      M3A        North York                         Parkwoods 
1      M4A        North York                  Victoria Village 
2      M5A  Downtown Toronto         Harbourfront ,Regent Park 
3      M6A        North York  Lawrence Heights ,Lawrence Manor 
4      M7A      Queen's Park                      Not assigned 


In [5]:
#read Geospatial_Coordinates.csv file which shows location of the postcodes
geospatial=pd.read_csv(r'/home/hatai/Geospatial_Coordinates.csv')
geospatial.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
#My plan is to join df dataframe and geospatial dataframe by using the index "Postcode" column.
#So, first let's set Postcode as an index for df.
indexPost_clean_df= df.set_index('Postcode')
indexPost_clean_df.head()

,Borough,Neighbourhood
Postcode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront ,Regent Park"
M6A,North York,"Lawrence Heights ,Lawrence Manor"
M7A,Queen's Park,Not assigned


In [7]:
#Second,renaming "Postal Code" to "Postcode"
geospatial.rename(columns={"Postal Code":"Postcode"},inplace=True)
geospatial.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
#For geospatial, using Postcode as index
new_geo=geospatial.set_index('Postcode')
new_geo.head()

,Latitude,Longitude
Postcode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [9]:
#join 2 dataframes, i.e. indexPost_clean_df and new_geo by using index column
result=indexPost_clean_df.join(new_geo)
result.head()

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Harbourfront ,Regent Park",43.654260,-79.360636
M6A,North York,"Lawrence Heights ,Lawrence Manor",43.718518,-79.464763
M7A,Queen's Park,Not assigned,43.662301,-79.389494


In [10]:
result.shape

(103, 4)

## Answer:  Second assignment

In [11]:
#Finally, reset index for the joined dataframe called,'result'.
result=result.reset_index()
print(result.shape)

(103, 5)


# Answer: Third assignment is below.
## Segmenting and Clustering Neighbourhoods in Toronta, Canada

In [12]:
#Prepare libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [13]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(result['Borough'].unique()),
        result.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toronto.
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>toronto_explorer</em>, as shown below.

In [14]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [15]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(result['Latitude'], result['Longitude'], result['Borough'], result['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

For illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Manhattan data.

In [16]:
# We will consider only Boroughs that have 'Toronto' in them.
borough_toronto = list(result.Borough.unique())

boroughs = []

for x in borough_toronto:
    if "toronto" in x.lower():
        boroughs.append(x)
        
boroughs

['Downtown Toronto', 'East Toronto', 'West Toronto', 'Central Toronto']

In [17]:
#Filtering only Toronto Boroughs
new_toronto = result[result['Borough'].isin(boroughs)].reset_index(drop=True)
print(new_toronto.shape)
new_toronto.head()

(38, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront ,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson ,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [18]:
toronto_data = new_toronto

In [19]:
#Map with markers only 5 Boroughs in Toronto
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(new_toronto['Latitude'], new_toronto['Longitude'], new_toronto['Borough'], new_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

#### You need to sigh up in Foursquare.com and obtain your client_id and client_secret.

In [20]:
# @hidden_cell
CLIENT_ID = '...' # your Foursquare ID
CLIENT_SECRET = '...' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:


In [21]:
#Explore first neighbourhood
new_toronto.loc[0, 'Neighbourhood']

'Harbourfront ,Regent Park '

In [22]:
neighbourhood_latitude = new_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = new_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = new_toronto.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Harbourfront ,Regent Park  are 43.6542599, -79.3606359.


In [23]:


#let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.
#First, let's create the GET request URL. Name your URL **url**.
LIMIT=100
radius=500


# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=RK3QIO50QEYU4ZUVDORMNFF5HTWL3IE4J51XDBRKKAQEN353&client_secret=4AK33Q4B5PR0WI0W4APLFEXEH1EOYSRWR2YUWNZXYCBN2TYX&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [24]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dbcd021a19793003818c1b5'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 51,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [25]:
#all the information is in the *items* key. Before we proceed, 
#let's borrow the **get_category_type** function from the Foursquare lab.
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [26]:
#clean the json and structure it into a pandas dataframe.
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

51 venues were returned by Foursquare.


In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
toronto_venues = getNearbyVenues(names=new_toronto['Neighbourhood'],
                                   latitudes=new_toronto['Latitude'],
                                   longitudes=new_toronto['Longitude']
                                  )

Harbourfront ,Regent Park 
Ryerson ,Garden District 
St. James Town 
The Beaches 
Berczy Park 
Central Bay Street 
Christie 
Adelaide ,King ,Richmond 
Dovercourt Village ,Dufferin 
Harbourfront East ,Toronto Islands ,Union Station 
Little Portugal ,Trinity 
The Danforth West ,Riverdale 
Design Exchange ,Toronto Dominion Centre 
Brockton ,Exhibition Place ,Parkdale Village 
The Beaches West ,India Bazaar 
Commerce Court ,Victoria Hotel 
Studio District 
Lawrence Park 
Roselawn 
Davisville North 
Forest Hill North ,Forest Hill West 
High Park ,The Junction South 
North Toronto West 
The Annex ,North Midtown ,Yorkville 
Parkdale ,Roncesvalles 
Davisville 
Harbord ,University of Toronto 
Runnymede ,Swansea 
Moore Park ,Summerhill East 
Chinatown ,Grange Park ,Kensington Market 
Deer Park ,Forest Hill SE ,Rathnelly ,South Hill ,Summerhill West 
CN Tower ,Bathurst Quay ,Island airport ,Harbourfront West ,King and Spadina ,Railway Lands ,South Niagara 
Rosedale 
Stn A PO Boxes 25 The Esplanad

In [30]:
print(toronto_venues.shape)
toronto_venues.head()

(1697, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront ,Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront ,Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront ,Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront ,Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Harbourfront ,Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [31]:
toronto_venues.groupby('Neighbourhood').count()
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
#Analyze Each Neighbourhood
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

There are 234 uniques categories.


,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Harbourfront ,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Harbourfront ,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [32]:
toronto_onehot.shape

(1697, 235)

In [33]:
#let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Adelaide ,King ,Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.03,0.0,0.0,0.010000,0.01,0.0,0.03,0.0,0.000000,0.000000,0.0,0.000000,0.030000,0.0,0.040000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.01,0.000000,0.01,0.020000,0.000000,0.0,0.01,0.02,0.010000,0.0,0.000000,0.050000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.010000,0.000000,0.070000,0.0,0.0,0.0,0.01,0.000000,0.000000,0.020000,0.000000,0.020000,0.0,0.000000,0.0,0.0,0.0,0.01,0.01,0.01,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.01,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.01,0.0,0.000000

In [34]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide ,King ,Richmond ----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.05
2          Bar  0.04
3   Steakhouse  0.03
4        Hotel  0.03


----Berczy Park ----
                venue  freq
0         Coffee Shop  0.07
1  Italian Restaurant  0.04
2  Seafood Restaurant  0.04
3            Beer Bar  0.04
4                Café  0.04


----Brockton ,Exhibition Place ,Parkdale Village ----
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3          Bakery  0.05
4       Pet Store  0.05


----Business Reply Mail Processing Centre 969 Eastern ----
                venue  freq
0  Light Rail Station  0.12
1         Pizza Place  0.06
2          Restaurant  0.06
3       Burrito Place  0.06
4          Smoke Shop  0.06


----CN Tower ,Bathurst Quay ,Island airport ,Harbourfront West ,King and Spadina ,Railway Lands ,South Niagara ----
              venue  freq
0   Airport Service  0.21
1    Airport Lounge  0.14
2  Airport Terminal  0.1

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide ,King ,Richmond",Coffee Shop,Café,Bar,Hotel,Thai Restaurant,Steakhouse,Restaurant,Bakery,Sushi Restaurant,Asian Restaurant
1,Berczy Park,Coffee Shop,Cheese Shop,Seafood Restaurant,Beer Bar,Italian Restaurant,Steakhouse,Cocktail Bar,Bakery,Farmers Market,Café
2,"Brockton ,Exhibition Place ,Parkdale Village",Café,Breakfast Spot,Coffee Shop,Gym,Climbing Gym,Burrito Place,Convenience Store,Stadium,Caribbean Restaurant,Restaurant
3,Business Reply Mail Processing Centre 969 East...,Light Rail Station,Gym / Fitness Center,Garden,Pizza Place,Recording Studio,Restaurant,Burrito Place,Skate Park,Smoke Shop,Brewery
4,"CN Tower ,Bathurst Quay ,Island airport ,Harbo...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Boutique,Sculpture Garden,Coffee Shop,Boat or Ferry,Airport Food Court,Airport


# Cluster Neighborhoods

In [37]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering =toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [38]:
toronto_data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront ,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson ,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Adelaide ,King ,Richmond",43.650571,-79.384568
8,M6H,West Toronto,"Dovercourt Village ,Dufferin",43.669005,-79.442259
9,M5J,Downtown Toronto,"Harbourfront East ,Toronto Islands ,Union Stat...",43.640816,-79.381752


In [39]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront ,Regent Park",43.654260,-79.360636,0,Coffee Shop,Pub,Park,Bakery,Café,Theater,Mexican Restaurant,Breakfast Spot,Yoga Studio,Greek Restaurant
1,M5B,Downtown Toronto,"Ryerson ,Garden District",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Bakery,Bubble Tea Shop,Bookstore,Fast Food Restaurant,Italian Restaurant,Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Hotel,Clothing Store,Bakery,Cosmetics Shop,Gastropub,Breakfast Spot,Italian Restaurant
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Neighborhood,Pub,Trail,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cheese Shop,Seafood Restaurant,Beer Bar,Italian Restaurant,Steakhouse,Cocktail Bar,Bakery,Farmers Market,Café


In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [41]:
#cluster1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Pub,Park,Bakery,Café,Theater,Mexican Restaurant,Breakfast Spot,Yoga Studio,Greek Restaurant
1,Downtown Toronto,0,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Bakery,Bubble Tea Shop,Bookstore,Fast Food Restaurant,Italian Restaurant,Restaurant
2,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Hotel,Clothing Store,Bakery,Cosmetics Shop,Gastropub,Breakfast Spot,Italian Restaurant
3,East Toronto,0,Health Food Store,Neighborhood,Pub,Trail,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
4,Downtown Toronto,0,Coffee Shop,Cheese Shop,Seafood Restaurant,Beer Bar,Italian Restaurant,Steakhouse,Cocktail Bar,Bakery,Farmers Market,Café
5,Downtown Toronto,0,Coffee Shop,Ice Cream Shop,Café,Italian Restaurant,Sandwich Place,Burger Joint,Salad Place,Bubble Tea Shop,Gym / Fitness Center,Bakery
6,Downtown Toronto,0,Grocery Store,Café,Park,Coffee Shop,Athletics & Sports,Convenience Store,Italian Restaurant,Diner,Baby Store,Candy Store
7,Downtown Toronto,0,Coffee Shop,Café,Bar,Hotel,Thai Restaurant,Steakhouse,Restaurant,Bakery,Sushi Restaurant,Asian Restaurant
8,West Toronto,0,Pharmacy,Supermarket,Bakery,Bank,Music Venue,Wine Shop,Brewery,Gym / Fitness Center,Park,Bar
9,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Scenic Lookout,Restaurant,Brewery,Fried Chicken Joint,Park,Italian Restaurant


In [42]:
#cluster2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Central Toronto,1,Playground,Restaurant,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [43]:
#cluster3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Central Toronto,2,Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
32,Downtown Toronto,2,Park,Playground,Trail,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [44]:
#cluster4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,3,Garden,Ice Cream Shop,Yoga Studio,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [45]:
#cluster5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Central Toronto,4,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


## Observation
Cluster1 seems to be a tourist attraction and many busy shops around there.
Cluster2, 3 and 4 seem to be more park and playground area than shops.